In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-12-31 04:11:57--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-12-31 04:11:57--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucebaccf7e0498ae9d6812fd5ea3.dl.dropboxusercontent.com/zip_download_get/Apc2vo96tex1e1vheeqeS88qnLpJn5EohVaoCUSq3FcB5OohXKgTNNSwEpm4IdiRh7HqBv00Lec4_zbkvnNtfb_CTV6EZBPzMa7DeolDeH0pXw [following]
--2020-12-31 04:11:58--  https://ucebaccf7e0498ae9d6812fd5ea3.dl.dropboxusercontent.com/zip_download_get/Apc2vo96tex1e1vheeqeS88qnLpJn5EohVaoCUSq3FcB5OohXKgTNNSwEpm4IdiRh7HqBv00Lec4_zbkv

In [3]:
!unzip AAAw1_JIzpuVvwteJCma0xMla?dl=0

Archive:  AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [4]:
df = pd.read_csv('train.csv')

In [5]:

df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [6]:
df.shape

(1306122, 3)

In [7]:
df['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [8]:
# Download the glove embedding

!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-12-31 04:12:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-31 04:12:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-31 04:12:58--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [9]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
df['length']=[len(x) for x in df['question_text']]
df['length'].max(), df['length'].min()

(1017, 1)

In [10]:
df.isnull().sum()

qid              0
question_text    0
target           0
dtype: int64

In [11]:
X = df['question_text'].values
y = df['target'].values

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(X)
vocab_size = len(t.word_index) + 1
#word to integers
encoded_word = t.texts_to_sequences(X)
# pad documents to a max length of 20 words
max_length = 20
padded_word = pad_sequences(encoded_word, maxlen=max_length, padding='post')
print(len(padded_word))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1306122


In [19]:
padded_word.shape

(1306122, 20)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_word, y, test_size=0.2, random_state=42)

In [21]:
embeddings_index = dict()
f = open("/content/glove.6B.100d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [22]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(222162, 100)


In [23]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=20, trainable=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           22216200  
_________________________________________________________________
flatten (Flatten)            (None, 2000)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2001      
Total params: 22,218,201
Trainable params: 2,001
Non-trainable params: 22,216,200
_________________________________________________________________
None


In [24]:
import keras
h = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30, batch_size=512,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)    ]
)

Epoch 1/30
2041/2041 [==============================] - 12s 5ms/step - loss: 0.1890 - accuracy: 0.9341 - val_loss: 0.1497 - val_accuracy: 0.9448
Epoch 2/30
2041/2041 [==============================] - 10s 5ms/step - loss: 0.1517 - accuracy: 0.9438 - val_loss: 0.1479 - val_accuracy: 0.9457
Epoch 3/30
2041/2041 [==============================] - 10s 5ms/step - loss: 0.1488 - accuracy: 0.9448 - val_loss: 0.1474 - val_accuracy: 0.9455
Epoch 4/30
2041/2041 [==============================] - 10s 5ms/step - loss: 0.1490 - accuracy: 0.9442 - val_loss: 0.1467 - val_accuracy: 0.9454
Epoch 5/30
2041/2041 [==============================] - 10s 5ms/step - loss: 0.1477 - accuracy: 0.9449 - val_loss: 0.1470 - val_accuracy: 0.9457
Epoch 6/30
2041/2041 [==============================] - 9s 5ms/step - loss: 0.1488 - accuracy: 0.9444 - val_loss: 0.1476 - val_accuracy: 0.9451
Epoch 7/30
2041/2041 [==============================] - 10s 5ms/step - loss: 0.1476 - accuracy: 0.9451 - val_loss: 0.1464 - val_acc